# 第7章 整数計画

## 7.1
>ナップサック問題 (7.7) を、整数計画のソルバーで解いてみよ。
>$$
>\begin{align*}
>\max \ &28x_1 + 33 x_2 + 10 x_3 + 45 x_4 + 32 x_5\\
>\mathrm{s.t.} \ &2x_1 + 3 x_2 + x_3 + 5x_4 + 4 x_5 \le 7\\
>&x_j \in \{0, 1\}, \ \ j = 1, \dots, 5.
>\end{align*}
>$$

In [1]:
from mip import Model, BINARY, xsum, maximize

In [2]:
m = Model()
x = [m.add_var(var_type=BINARY) for _ in range(5)]
c = [28, 33, 10, 45, 32]
a = [2, 3, 1, 5, 4]

m.objective = maximize(xsum(c[i]*x[i] for i in range(5)))
m += xsum(a[i]*x[i] for i in range(5)) <= 7

m.optimize()

from IPython.display import clear_output
clear_output()

In [3]:
print("Status:", m.status)
print("Value:", m.objective_value)
print("Solution: x =", [z.x for z in m.vars])

Status: OptimizationStatus.OPTIMAL
Value: 73.0
Solution: x = [1.0, 0.0, 0.0, 1.0, 0.0]


## 7.2
>水曜3限の $n$ 個の講義科目を $m$ 個の教室（$m > n$）に割り当てる問題を考える.
>講義の受講者数を $d_1, \dots, d_n$ とおき, 教室の収容人数を$r_1, \dots, r_m$ とおく.
>受講者数が収容人数を超える教室の数を最小にする問題を, 整数計画問題として定式化せよ.
>ただし, $n$ 個すべての講義をいずれかの教室で開講しなければならない.
>また, 一つの教室で複数の講義を開講することはできない.

$x = (x_{ij})_{i=1,\dots,n,\ j=1,\dots,m}$ は, 講義 $i$ が教室 $j$ を使うかどうかを表す 0-1 変数.  
$y_i \in \{0, 1\}$ は, 講義 $i$ について受講者数が教室の収容人数を超えるかどうかを表す 0-1 変数とする.  
$M$ を十分大きな正整数として, この問題は次の整数計画問題として定式化できる.

$$
\begin{align*}
    \min_{x, y} \ & \sum_{i=1}^n y_i\\
    \mathrm{s.t.} \ \ &\sum_{j = 1}^m x_{ij} = 1, \ \forall i = 1, \dots, n\\
    &\sum_{i=1}^n x_{ij} \le 1, \ \forall j = 1, \dots, m\\
    &My_i \ge d_i - \sum_{j = 1}^mx_{ij}r_j, \ \forall i = 1, \dots, n\\
    &x_{ij}, y_i \in \{0, 1\}.
\end{align*}
$$

## 7.3
>非階層的クラスタリングの一つとして, クラスターの半径のうち最大のものを最小化する問題を考える.
>ただし, クラスターの半径とは, そのクラスターに属するすべての点を含む最小の円の半径であると定義する.
>この問題を, 整数制約付き2次錐計画問題として定式化せよ.

データ点の集合は $s_1, \dots, s_m$ とする.   
各クラスターの中心を表す変数を $c_1, \dots, c_k$, 各データ点 $\ell$ がクラスター $i$ に含まれることを示す 0-1 変数を $y_{i\ell}$ とする.  
このとき, この問題は以下の最適化問題として定式化される:
$$
\begin{align*}
    \min_{z, c, y} \ &z\\
    \mathrm{s.t.} \ \ &\sum_{i=1}^k y_{i\ell} = 1, \ \forall \ell = 1,\dots, m\\
    &z + M(1-y_{i\ell}) \ge \|s_\ell - c_i\|, \ \forall i, j,\\
    &y_{i\ell} \in \{0, 1\}, \forall i, j.
\end{align*}
$$

## 7.4
>$k$-meansクラスタリング法（6.3.2節）では, クラスターの重心からデータ点の距離の2乗和を最小化する問題
>$$ \min \sum_{i=1}^k \sum_{s_\ell \in C_i} \rho(s_\ell, \mu_i)^2  \tag{7.27} $$
>を考えた. ただし,クラスター $C_i$ の重心 $\mu_i$ は, $C_i$ に属する点からの距離の2乗和が最小の点である:
>$$ \mu_i = \argmin_{\mu' \in \mathbb{R}^n}\sum_{s_\ell \in C_i}\rho(s_\ell, \mu').$$
>問題 (7.27) と等価な問題として, 目的関数が線形関数であり, 制約が線形制約, 2次錐制約, 整数制約の3種類のみであるような問題を定式化せよ.

変数 $y_{i\ell}$ はデータ点 $s_\ell$ がクラスター $i$ に属すことを表す 0-1 変数, $M$ は十分大きな定数とする.  
上記の問題は以下の最適化問題と等価である.
$$
\begin{align*}
\min &\sum_{\ell = 1}^m z_\ell\\
\mathrm{s.t.} \ \ &\sum_{i = 1}^m y_{i\ell} = 1, \ \forall \ell = 1, \dots, m,\\
&z_{\ell i} + M(1-y_{\ell i}) + 1 \ge \left\|\begin{bmatrix}z_{\ell i} + M(1-y_{\ell i}) - 1\\2(s_\ell - \mu_i)\end{bmatrix}\right\|_2, \ \forall \ell, i\\
&y_{\ell i} \in \{0, 1\}, \forall \ell, i
\end{align*}
$$
ただし, 2つ目の制約式は
$$z_\ell + M (1-y_{i\ell}) \ge \|z_\ell - \mu_i\|_2^2$$
と同値である.

## 7.5
>文書要約に関する最適化問題を, 整数計画問題として定式化せよ.

文書に含まれる文を $i = 1, \dots, n$ とする.   
例えば (6.15) にあるように, 要約に含む文の文全体との類似度 $r_i$ を最大化する問題を考えるなら, 以下のように整数計画問題として定式化できる.  
ただし, $m$ は要約に含められる文の数の制約.
$$
\begin{align*}
    \max_{(x_i)_{i = 1,\dots,n}} \ &\sum_{i=1}^n x_i r_i\\
    \mathrm{s.t.} \ \ &\sum_{i=1}^n x_i \le m,\\
    &x_i \in \{0, 1\}, \ \forall i = 1, \dots, n.
\end{align*}
$$